# Pipeline

### read_data(file_path)
- read data to df

### concat_user_input_and_convert_to_csr(df, user_input): 
- df: output df from read_data
- user_input: list of courses the user provide
- Append user input to the df and convert the df into csr. 
- Returns csr matrix, and course list.

### svd_based(uc_mat, courses, user_idx = -1, num_factors=80, top_n = 10, random_state=42):
- uc_mat: csr matrix (1st output of concat_user_input_and_convert_to_csr)
- courses: course list (2nd output of concat_user_input_and_convert_to_csr)
- num_factors: hyperparam of svd
- top_n: num of recommendation
- Returns top_n recommended courses

---------

# This notebook
* Try three different collaborative filterings: Memory, SVD, NMF
* Speed: SVD - Memory - NMF (fast to slow)
* RMSE: SVD - NMF - Memory (low to high)


* Winner: SVD

## Note
* Parameter tuning (SVD: num_factors)

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

## Load the data and convert into sparse matrix

In [12]:
def read_data(file_path):
    from scipy.sparse import csr_matrix
    
    # Read pickle
    stu = pd.read_pickle(file_path)
    
    # Num of dictionaries in the file
    n = len(stu)
    
    # Initialize df
    df = pd.DataFrame()
    
    # Loop through dicts and concat
    for i in range(n):
        stu[i]['student_id'] = i
        df = pd.concat([df,stu[i]])
    
    return df

def concat_user_input_and_convert_to_csr(df, user_input):
    # df = user dataset we have
    # user_input: courses that user provide

    # User input to df and concat
    user_df = pd.DataFrame()
    user_df['Subject/Catalog'] = user_input
    user_df['Rating'] = 1
    user_df['student_id'] = df['student_id'].max()+1

    df = pd.concat([df, user_df])
    # Fix .0 issues -- Some courses has .0 at the end (e.g. AAS 101.0), remove .0
    df.loc[df['Subject/Catalog'].str.contains('\.') == True, 'Subject/Catalog'] = df.loc[df['Subject/Catalog'].str.contains('\.') == True, 'Subject/Catalog'].str.split('\.').str[0]
    
    # Pivot table to stu - course
    df = pd.pivot_table(df, values = 'Rating', index = 'student_id', columns = 'Subject/Catalog')
    
    # Fillna
    df = df.fillna(0)
    
    # df to sparse matrix
    csr = csr_matrix(df)
    
    # Store course names
    courses = list(df.columns)
    
    # Get matched user by cos sim -- uncomment this if we use this information
    #matched_user_idx = np.argsort(cosine_similarity(csr)[-1])[-2] #Best match of stu_idx -1 
    
    return csr, courses #matched_user_idx

In [46]:
df = read_data('student_profiles.pickle')

,Subject/Catalog,Course List Description,Credits,Course Title,Rating,student_id
0,MATH 295,CS-LSA major declaration Cal I,4.0,Honors Math I,4.021522,0
1,MATH 116,CS-LSA major declaration Cal II,4.0,Calculus II,3.503458,0
2,IOE 265,Core Probability,3.0,Prob&Stat for Eng,2.513271,0
3,EECS 482,Upper Level CS Technical Electives (ULCS),4.0,Intro Oper System,2.275309,0
4,EECS 590,Upper Level CS Technical Electives (ULCS),4.0,Adv Prog Lang,2.329611,0
...,...,...,...,...,...,...
28,MEM 210,SS,4.0,Early Med Europe,3.372078,4999
29,HISTOR 340,SS,4.0,From Genghis Khan to the Taliban: Modern Cent...,3.164601,4999
30,HISTOR 394,HU,3.0,History of the Turkish Republic,3.941316,4999
31,AMCUL 392,HU,3.0,"Mad, Bad, and Sad: Mental Health in the U.S.",3.627988,4999


In [58]:
subcat = df['Subject/Catalog'].unique()

## Matrix-based collaborative filtering

In [38]:
from sklearn.decomposition import TruncatedSVD

def svd_based(uc_mat, courses, num_factors=80, top_n = 10, random_state=42):
    # SVD based collaborative filtering
    
    # user_idx = -1
    user_idx = -1
    
    param_grid = {'n_components' : [60, 65, 70, 75, 80, 85]}
    
    # SVD
    svd = TruncatedSVD(n_components = num_factors, random_state = random_state)
    
    # Transform X
    X_transformed = svd.fit_transform(uc_mat)
    
    # Calculate r
    r = np.matmul(X_transformed, svd.components_)
    
    # Pred rating
    pred_rating = r[user_idx,:]#.reshape(1,-1)
    
    
    # Matched user vector
    user_vec = uc_mat[user_idx].toarray().flatten()
    
    
    # User's favorite course indices
    user_fav_idx = np.argsort(-user_vec)
    
    # User's favorite courses
    #user_fav = [courses[i] for i in user_fav_idx[:top_n]]
    #print(f"Top {top_n} favorite courses:")
    #print(user_fav)

    
    # Flatten pred_rating
    pred_rating = np.asarray(pred_rating).flatten()
    
    # Argsort the rating except the courses taken by the user
    recommend_index = np.argsort(-pred_rating[user_vec == 0])
    
    # Recommend top courses
    top_courses = [courses[i] for i in recommend_index[:top_n]]
    
    print(f"Top {top_n} recommended courses:")
    print(top_courses)

    print()

    # RMSE between true & predicted. Uncomment the next two lines to see RMSE
    rmse = np.sqrt(np.sum((user_vec[user_vec != 0] - pred_rating[user_vec != 0]) ** 2))
    print(f"RMSE = {rmse}")
    
    return top_courses

In [39]:
%%time
pred = svd_based(csr, courses)

Top 10 recommended courses:
['EECS 581', 'EECS 385', 'EECS 482', 'RELIGIO 310', 'EECS 473', 'EECS 487', 'ENVIRO 167', 'EECS 490', 'EECS 280', 'SA 315']

RMSE = 4.320117968276276
CPU times: user 793 ms, sys: 117 ms, total: 911 ms
Wall time: 418 ms


## Calculate avg RMSE of SVD with diff n_factors

In [63]:
subcat = df['Subject/Catalog'].unique()

In [81]:
import numpy as np
def svd_tune(df, num_factors, top_n = 10, random_state=42):
    # SVD based collaborative filtering
    
    # user_idx = -1
    user_idx = -1
    rmse_list = []
    for i in range(100):
        user_input = np.random.choice(subcat, size = 10, replace = False)
        uc_mat, courses = concat_user_input_and_convert_to_csr(df, user_input)

        # SVD
        svd = TruncatedSVD(n_components = num_factors, random_state = random_state)

        # Transform X
        X_transformed = svd.fit_transform(uc_mat)

        # Calculate r
        r = np.matmul(X_transformed, svd.components_)

        # Pred rating
        pred_rating = r[user_idx,:]#.reshape(1,-1)


        # Matched user vector
        user_vec = uc_mat[user_idx].toarray().flatten()


        # User's favorite course indices
        user_fav_idx = np.argsort(-user_vec)

        # User's favorite courses
        #user_fav = [courses[i] for i in user_fav_idx[:top_n]]
        #print(f"Top {top_n} favorite courses:")
        #print(user_fav)


        # Flatten pred_rating
        pred_rating = np.asarray(pred_rating).flatten()

        # Argsort the rating except the courses taken by the user
        recommend_index = np.argsort(-pred_rating[user_vec == 0])

        # Recommend top courses
        top_courses = [courses[i] for i in recommend_index[:top_n]]


        # RMSE between true & predicted. Uncomment the next two lines to see RMSE
        rmse_list.append(np.sqrt(np.sum((user_vec[user_vec != 0] - pred_rating[user_vec != 0]) ** 2)))
    avg_rmse = np.average(rmse_list)
    #print(f"RMSE = {avg_rmse}")
    
    return avg_rmse

In [62]:
avg_rmse_list = []
for num in [50, 55, 60, 65, 70, 75, 80, 85, 90]:
    avg_rmse = svd_tune(df, num, top_n = 10, random_state=42)
    avg_rmse_list.append(avg_rmse)
    

RMSE = 4.364242197081958
RMSE = 4.357827404798507
RMSE = 4.359784457612434
RMSE = 4.346068447475665
RMSE = 4.337082405850965
RMSE = 4.330620854141026
RMSE = 4.320117968276278
RMSE = 4.324448355403818
RMSE = 4.30260091539495


## SVD hyperparameter tuning 

In [67]:
#pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=66c6540c73156f630062a7533f31207b79a0e40505d64949e6d2badf702820a3
  Stored in directory: /Users/Yeseul/Library/Caches/pip/wheels/37/fa/19/f93e793d3944567a60b3ab93b446cf7370cc82c60c1d1c613f
Successfully built bayesian-optimization
Note: you may need to restart the kernel to use updated packages.


In [90]:
from bayes_opt import BayesianOptimization

def BO_func(n_components):
    avg_rmse = svd_tune(df, int(n_components), top_n = 10, random_state=42)
    return -avg_rmse

In [91]:
%%time
tuning_params = { 
  "n_components": (100, 150) }

optimizer = BayesianOptimization(
  f = BO_func,
  pbounds = tuning_params,
  verbose = 5,
  random_state = 42
 )

optimizer.maximize(
  init_points = 4,
  n_iter = 3 
 )

|   iter    |  target   | n_comp... |
-------------------------------------
|  1        | -3.087    |  118.7    |
|  2        | -3.061    |  147.5    |
|  3        | -3.087    |  136.6    |
|  4        | -3.083    |  129.9    |
|  5        | -3.067    |  146.6    |
|  6        | -3.083    |  150.0    |
|  7        | -3.068    |  144.2    |
CPU times: user 32min 52s, sys: 5min 50s, total: 38min 42s
Wall time: 31min 21s


In [93]:
tuning_params = { 
  "n_components": (50, 500) }


from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(
  f = BO_func,
  pbounds = tuning_params,
  verbose = 5,
  random_state = 42
 )

optimizer.maximize(
  init_points = 4,
  n_iter = 3 
 )

|   iter    |  target   | n_comp... |
-------------------------------------
|  1        | -3.024    |  218.5    |
|  2        | -2.86     |  477.8    |
|  3        | -2.903    |  379.4    |
|  4        | -2.96     |  319.4    |
|  5        | -2.848    |  500.0    |
|  6        | -2.814    |  498.8    |
|  7        | -2.849    |  494.9    |


## Memory-based collaborative filtering

In [32]:
import scipy.sparse
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
uc_mat = csr

def memory_based(uc_mat, user_idx, top_n):
    # Memory-based collaborative filtering
    
    # Normalize user-course matrix
    norm_uc_mat = uc_mat - uc_mat.mean(axis = 1).reshape(-1,1)
    
    # Cosine similarity of raw matrix
    cossim = cosine_similarity(uc_mat)
    
    # Calculate the numerator of V-hat(aj)
    v_num = np.dot(cossim[user_idx,:], norm_uc_mat)
    
    # Sum of V-hat(aj) numerator
    v_num_sum = v_num.sum(axis = 0)
    
    # V_hat(aj)
    v_hat = v_num_sum/(cossim[user_idx,:].sum())
    
    #Predicted rating of each course
    pred_rating = np.add(v_hat, uc_mat.mean(axis = 1).reshape(-1,1)[user_idx])
    
    # User vector
    user_vec = uc_mat[user_idx].toarray().flatten()
    
    # User's favorite course indices
    user_fav_idx = np.argsort(-user_vec)
    
    # User's favorite courses
    user_fav = [courses[i] for i in user_fav_idx[:top_n]]
    print(f"Top {top_n} favorite courses:")
    print(user_fav)
    
    # Flatten pred_rating
    pred_rating = np.asarray(pred_rating).flatten()
    
    
    # Argsort the rating except the courses taken by the user
    recommend_index = np.argsort(-pred_rating[user_vec == 0])
    
    # Recommend top courses
    top_courses = [courses[i] for i in recommend_index[:top_n]]
    
    print(f"Top {top_n} recommended courses:")
    print(top_courses)

    print()

    # RMSE between true & predicted
    rmse = np.sqrt(np.sum((user_vec[user_vec != 0] - pred_rating[user_vec != 0]) ** 2))
    print(f"RMSE = {rmse}")
    
    return top_courses



In [33]:
%%time
pred = memory_based(csr, -1, 10)

Top 10 favorite courses:
['ENGLIS 220', 'EART 156', 'NATIVEA 310', 'EART 255', 'WG 211', 'ASIA 243', 'ASIA 244', 'MAT 216', 'ASIANLA 266', 'NATIVEA 422']
Top 10 recommended courses:
['EEC 183', 'EE 484', 'EEC 280', 'EE 483', 'MAT 295', 'MAT 403', 'MAT 296', 'MAT 286', 'MAT 327', 'MAT 297']

RMSE = 4.873693740931147
CPU times: user 1.02 s, sys: 444 ms, total: 1.46 s
Wall time: 1.68 s


## NMF-based collaborative filtering

In [36]:
from sklearn.decomposition import NMF

def nmf_based(uc_mat, user_idx, num_factors, top_n = 10, random_state=42):
    # NMF based collaborative filtering
    
    # NMF
    nmf = NMF(n_components = num_factors, init = 'nndsvd', max_iter = 300, random_state = random_state)
    
    # Transform X
    X_transformed = nmf.fit_transform(uc_mat)
    
    # Calculate r
    r = np.matmul(X_transformed, nmf.components_)
    
    # Predict rating
    pred_rating = r[user_idx,:].reshape(1,-1)
    
    
    # User vector
    user_vec = uc_mat[user_idx].toarray().flatten()
    
    # User's favorite course indices
    user_fav_idx = np.argsort(-user_vec)
    
    # User's favorite courses
    user_fav = [courses[i] for i in user_fav_idx[:top_n]]
    print(f"Top {top_n} favorite courses:")
    print(user_fav)
    
    # Flatten pred_rating
    pred_rating = np.asarray(pred_rating).flatten()
    
    
    # Argsort the rating except the courses taken by the user
    recommend_index = np.argsort(-pred_rating[user_vec == 0])
    
    # Recommend top courses
    top_courses = [courses[i] for i in recommend_index[:top_n]]
    
    print(f"Top {top_n} recommended courses:")
    print(top_courses)

    print()

    # RMSE between true & predicted
    rmse = np.sqrt(np.sum((user_vec[user_vec != 0] - pred_rating[user_vec != 0]) ** 2))
    print(f"RMSE = {rmse}")
    
    return top_courses



In [37]:
%%time
pred = nmf_based(csr, -1, 80, )

Top 10 favorite courses:
['ENGLIS 220', 'EART 156', 'NATIVEA 310', 'EART 255', 'WG 211', 'ASIA 243', 'ASIA 244', 'MAT 216', 'ASIANLA 266', 'NATIVEA 422']
Top 10 recommended courses:
['EEC 183', 'LIN 393', 'EECS 487', 'EECS 482', 'EECS 581', 'EECS 491', 'EECS 574', 'RELIGIO 310', 'EECS 543', 'EECS 478']

RMSE = 4.346884211155051
CPU times: user 31.4 s, sys: 614 ms, total: 32.1 s
Wall time: 35.9 s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
